You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: 

Title: The Future of Artificial Intelligence: Trends, Players, and News

Outline:

I. Introduction
- Definition of Artificial Intelligence (AI)
- Importance of AI in various industries
- Brief overview of the latest trends in AI

II. Latest Trends in AI
- Advancements in machine learning and deep learning
- Rise of AI-powered chatbots and virtual assistants
- Increased focus on ethical AI practices
- Expansion of AI in healthcare, finance, and automotive industries

III. Key Players in the AI

I now can give a great answer

Final Answer: 

# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction

Artificial Intelligence (AI) is a rapidly evolving technology that simulates human intelligence to perform tasks such as speech recognition, decision-making, and visual perception. Its importance in various industries cannot be overstated, as AI has the potential to revolutionize healthcare, finance, automotive, and many other sectors. The latest trends in AI showcase advancements in machine learning and deep learning, the rise of AI-powered chatbots and virtual assistants, an increased focus on ethical AI practices, and the expansion of AI across different industries.

## Latest Trends in AI

One of the most exciting trends in AI is the advancements in machine learning and deep learning algorithms, which have significantly improved the capabilities of AI systems. This has led to the rise of AI-powered chatbots and virtual assistants that are becoming incr

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction

Artificial Intelligence (AI) is a rapidly evolving technology that simulates human intelligence to perform tasks such as speech recognition, decision-making, and visual perception. Its importance in various industries cannot be overstated, as AI has the potential to revolutionize healthcare, finance, automotive, and many other sectors. The latest trends in AI showcase advancements in machine learning and deep learning, the rise of AI-powered chatbots and virtual assistants, an increased focus on ethical AI practices, and the expansion of AI across different industries.

## Latest Trends in AI

One of the most exciting trends in AI is the advancements in machine learning and deep learning algorithms, which have significantly improved the capabilities of AI systems. This has led to the rise of AI-powered chatbots and virtual assistants that are becoming increasingly integrated into everyday life. Moreover, there is a growing emphasis on ethical AI practices to ensure that AI technologies are developed and deployed responsibly. AI is also making significant strides in healthcare, finance, and the automotive industry, where it is being used to enhance decision-making processes and improve overall efficiency.

## Key Players in the AI Industry

Leading tech companies such as Google, Amazon, and Microsoft are heavily investing in AI research and development, driving innovation in the field. Additionally, startups are making significant contributions to AI by bringing fresh perspectives and ideas to the table. Research institutions and universities are also at the forefront of AI research, pushing the boundaries of what is possible with AI technology.

## Noteworthy News in AI

Recent breakthroughs in AI technology, such as the development of self-learning algorithms and autonomous systems, are reshaping the way we interact with AI. However, the impact of AI on the job market and society as a whole is a topic of concern, with questions arising about the displacement of human workers by AI systems. Regulatory developments related to AI implementation are also being closely monitored to ensure that AI technologies are developed and deployed ethically and responsibly.

In conclusion, Artificial Intelligence is a transformative technology that is shaping the future of various industries. By staying informed about the latest trends, key players, and news in the AI industry, individuals can make informed decisions about how to leverage AI for their organizations and prepare for the future of AI technology. Stay informed by subscribing to our newsletter and following us on social media for regular updates on AI advancements.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "Nifty50 and Sensex in Indian Stock Market"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Nifty50 and Sensex in Indian Stock Market.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan:

Title: Understanding Nifty50 and Sensex in the Indian Stock Market: Key Trends and Analysis

Introduction:
- Brief explanation of the Nifty50 and Sensex indices
- Importance of tracking these indices in the Indian stock market
- Overview of the latest trends and key players in the market

Key Points:
1. Latest Trends in Nifty50 and Sensex:
- Analysis of recent market performance
- Impact of global economic factors on the Indian stock market
- Noteworthy fluctuations or pattern

I now can give a great answer

Final Answer:
# Understanding Nifty50 and Sensex in the Indian Stock Market: Key Trends and Analysis

## Introduction

The Nifty50 and Sensex are two prominent stock market indices in India that play a crucial role in tracking the overall performance of the Indian stock market. The Nifty50 index represents the performance of the top 50 companies listed on the National Stock Exchange (NSE), while the Sensex tracks the 30 largest and most actively traded stocks on the Bombay Stock Exchange (BSE). Understanding these indices is essential for investors, traders, and finance professionals to make informed decisions and capitalize on market opportunities. In this article, we will delve into the latest trends and analysis of the Nifty50 and Sensex, highlighting key players and providing insights into the Indian stock market.

## Latest Trends in Nifty50 and Sensex

In recent times, the Indian stock market has witnessed significant fluctuations driven by a combin

In [14]:
Markdown(result)

# Understanding Nifty50 and Sensex in the Indian Stock Market: Key Trends and Analysis

## Introduction

The Nifty50 and Sensex are two prominent stock market indices in India that play a crucial role in tracking the overall performance of the Indian stock market. The Nifty50 index represents the performance of the top 50 companies listed on the National Stock Exchange (NSE), while the Sensex tracks the 30 largest and most actively traded stocks on the Bombay Stock Exchange (BSE). Understanding these indices is essential for investors, traders, and finance professionals to make informed decisions and capitalize on market opportunities. In this article, we will delve into the latest trends and analysis of the Nifty50 and Sensex, highlighting key players and providing insights into the Indian stock market.

## Latest Trends in Nifty50 and Sensex

In recent times, the Indian stock market has witnessed significant fluctuations driven by a combination of domestic and global economic factors. The performance of the Nifty50 and Sensex is closely linked to global market trends, geopolitical events, and macroeconomic indicators. Analysts have been closely monitoring the impact of the COVID-19 pandemic, geopolitical tensions, and government policies on the Indian stock market. Noteworthy fluctuations in the indices have reflected investor sentiments and market volatility, creating both challenges and opportunities for market participants.

## Key Players in the Indian Stock Market

The Nifty50 and Sensex comprise some of the largest and most influential companies in India across various sectors such as IT, banking, healthcare, and energy. Companies like Reliance Industries, Infosys, HDFC Bank, and Tata Consultancy Services are among the top performers on these indices, driving market trends and influencing investor decisions. Sector-wise performance analysis reveals the strengths and weaknesses of different industries, providing valuable insights for portfolio diversification and investment strategies. Understanding the key players in the Indian stock market is essential for identifying potential investment opportunities and mitigating risks.

## Noteworthy News Updates

Staying updated on the latest developments in the Indian stock market is crucial for making informed investment decisions. Regulatory changes, mergers, acquisitions, and corporate earnings reports can significantly impact stock prices and market dynamics. Analyst predictions and market forecasts offer valuable insights into future trends and investment opportunities. Investors, traders, and finance professionals rely on reliable sources such as financial news websites and market research reports to stay ahead of market trends and capitalize on emerging opportunities.

## Conclusion

In conclusion, tracking the Nifty50 and Sensex indices provides valuable insights into the dynamics of the Indian stock market and helps investors navigate through market uncertainties. By analyzing key trends, understanding market players, and staying informed about market news, investors can make informed decisions and optimize their investment portfolios. It is essential for the audience to stay proactive in their market analysis, seek advice from financial advisors, and continue learning about investment opportunities in the ever-evolving Indian stock market landscape. Stay informed, stay proactive, and seize the opportunities that the Indian stock market presents.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).